In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.pyplot import imshow

mat = scipy.io.loadmat('data_EEG_AI.mat')
alphabets = list(map(chr, range(65, 91)))
alabel = mat["label"][:,0]
label_dict = {}
for i in range(len(alabel)):
	if alabel[i] not in label_dict:
		label_dict[alabel[i]] = [i,7800]
	else:
		label_dict[alabel[i]][1] = i
print(label_dict)

In [ ]:
print(mat.keys())

In [ ]:
amat = pd.Series(mat)
amat = amat[['channel_labels', 'data', 'label', 'time_points']]
print(amat)

In [ ]:
df = []
colname = ['sample_no','channel_labels','label','time_points','data']
for i in range(7800*24):
	row_content = []
	row_content.append(i//24)
	row_content.append(mat["channel_labels"][i%24][0][0])
	row_content.append(i//(300*24))
	row_content.append(list(range(-200,3001,4)))
	row_content.append(mat['data'][i%24][:,i//24])
	df.append(row_content)
			
df = pd.DataFrame(df,columns=colname)
#df.to_csv("EEG_sorted.csv",index=False) 
df.head()

In [ ]:
for sample in range(1):
	plt.figure(figsize=(16,9))
	plt.title(f"sample = {sample}, label = {alphabets[mat['label'][sample][0]-1]}")
	plt.ylabel("potential")
	plt.xlabel("time after stimulus")
	for channel in range(len(mat['data'])):
		all = []
		for time in range(len(mat['data'][channel])):
			all.append(mat['data'][channel][time][sample])
		plt.plot(mat['time_points'][:,0],all, label=mat['channel_labels'][channel][0][0])
	plt.legend(loc='best')
	plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
import seaborn as sns

# overall
overall_avg = []
for channel in range(len(mat['data'])):
    all = []
    for time in mat['data'][channel]:
        all.append(sum(time)/len(time))
    overall_avg.append(all)

#each alphabet avg
alpha_avg = []
for i in label_dict:
    avg = []
    for channel in range(len(mat['data'])):
        all = []
        for time in mat['data'][channel]:
            all.append(sum(time[label_dict[i][0]:label_dict[i][1]])/300)
        avg.append(all)
    alpha_avg.append(avg)

print(np.array(overall_avg).shape)
print(np.array(alpha_avg).shape)

alpha_avg_mse = []
for i in alpha_avg:
    alpha_avg_mse.append(mean_squared_error(overall_avg, i))
print(alpha_avg_mse)

#heatmap
inter_alpha = []
for i in alpha_avg:
    row = []
    for j in alpha_avg:
        row.append(mean_squared_error(i, j))
    inter_alpha.append(row)

plt.figure(figsize = (10,8))
sns.heatmap(np.array(inter_alpha), xticklabels=alphabets, yticklabels=alphabets)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
import seaborn as sns

a = []
for i in mat['channel_labels']:
    a.append(i[0][0])

# overall
overall_avg = []
for channel in range(len(mat['data'])):
    all = []
    for time in mat['data'][channel]:
        all.append(sum(time)/len(time))
    overall_avg.append(all)
overall_channel_avg = np.mean(np.array(overall_avg), axis= 0)
    

#each channel avg
channel_avg = []
for channel in range(len(mat['data'])):
    all = []
    for time in mat['data'][channel]:
        all.append(sum(time)/len(time))
    channel_avg.append(all)

print(np.array(overall_channel_avg).shape)
print(np.array(channel_avg).shape)

channel_avg_mse = []
for i in channel_avg:
    channel_avg_mse.append(mean_squared_error(overall_channel_avg, i))
print(channel_avg_mse)

#heatmap
inter_channel = []
for i in channel_avg:
    row = []
    for j in channel_avg:
        row.append(mean_squared_error(i, j))
    inter_channel.append(row)

plt.figure(figsize = (10,8))
sns.heatmap(np.array(inter_channel), xticklabels=a, yticklabels=a, vmax=2)
plt.show()

In [ ]:
all_min = []
for channel in range(len(mat['data'])):
	channel_min = []
	for time in mat['data'][channel]:
		channel_min.append(min(time))
	all_min.append(channel_min)
print(all_min)
plt.figure(figsize=(16,9))
for c in range(24):
	plt.plot(mat['time_points'][:,0], all_min[c], label=mat['channel_labels'][channel][0][0])
plt.legend(loc='best')
plt.show()

In [ ]:
all_avg = []
for channel in range(len(mat['data'])):
	channel_avg = []
	for time in mat['data'][channel]:
		channel_avg.append(sum(time)/7800)
	all_avg.append(channel_avg)
print(all_avg)

plt.figure(figsize=(16,9))
for c in range(24):
	plt.plot(mat['time_points'][:,0], all_avg[c], label=mat['channel_labels'][c][0][0])
plt.legend(loc='best')
plt.show()

In [ ]:
#each alphabet avg
for i in label_dict:
	plt.figure(figsize=(16,9))
	plt.title(f"sample {label_dict[i][0]} to {label_dict[i][1]} average, label = {alphabets[i-1]}")
	plt.ylabel("potential")
	plt.xlabel("time after stimulus")
	for channel in range(len(mat['data'])):
		all = []
		for time in mat['data'][channel]:
			all.append(sum(time[label_dict[i][0]:label_dict[i][1]])/300)
		plt.plot(mat['time_points'][:,0],all, label=mat['channel_labels'][channel][0][0])
	plt.legend(loc='best')
	plt.show()

In [ ]:
#each alpahbet avg - min for each time frame
for i in label_dict:
	plt.figure(figsize=(16,9))
	plt.title(f"sample {label_dict[i][0]} to {label_dict[i][1]} average, label = {alphabets[i-1]}")
	plt.ylabel("potential")
	plt.xlabel("time after stimulus")
	for channel in range(len(mat['data'])):
		all = []
		for time in range(len(mat['data'][channel])):
			all.append(sum(mat['data'][channel][time][label_dict[i][0]:label_dict[i][1]])/300-all_min[channel][time])
		plt.plot(mat['time_points'][:,0],all, label=mat['channel_labels'][channel][0][0])
	plt.legend(loc='best')
	plt.show()

In [ ]:
#each alpahbet avg - total avg for each time frame
for i in label_dict:
	plt.figure(figsize=(16,9))
	plt.title(f"sample {label_dict[i][0]} to {label_dict[i][1]} average, label = {alphabets[i-1]}")
	plt.ylabel("potential")
	plt.xlabel("time after stimulus")
	for channel in range(len(mat['data'])):
		all = []
		for time in range(len(mat['data'][channel])):
			all.append(sum(mat['data'][channel][time][label_dict[i][0]:label_dict[i][1]])/300-all_avg[channel][time])
		plt.plot(mat['time_points'][:,0],all, label=mat['channel_labels'][channel][0][0])
	plt.legend(loc='best')
	plt.show()

In [ ]:
#each alphabet avg with sigmoid
def sig(x):
	return 1/(1 + np.exp(-x))
for i in label_dict:
	plt.figure(figsize=(16,9))
	plt.title(f"sample {label_dict[i][0]} to {label_dict[i][1]} average, label = {alphabets[i-1]}")
	plt.ylabel("potential")
	plt.xlabel("time after stimulus")
	for channel in range(len(mat['data'])):
		all = []
		for time in range(len(mat['data'][channel])):
			all.append(sum(mat['data'][channel][time][label_dict[i][0]:label_dict[i][1]])/300)
		plt.plot(mat['time_points'][:,0], list(map(sig, all)), label=mat['channel_labels'][channel][0][0])
	plt.legend(loc='best')
	plt.show()

In [ ]:
def sig(x):
	return 1/(1 + np.exp(-x))
for sample in [5,301,601]:
	plt.figure(figsize=(16,9))
	plt.title(f"sample{sample}")
	bw = []
	for channel in range(len(mat['data'])):
		all = []
		for time in mat['data'][channel]:
			all.append(time[sample])
		bw.append(list(map(sig, all)))
	im = Image.fromarray(np.array(bw), mode='L')
	imshow(np.asarray(im))
	plt.show()

In [ ]:
#each alphabet avg with min max normalization

for i in label_dict:
	plt.figure(figsize=(16,9))
	plt.title(f"sample {label_dict[i][0]} to {label_dict[i][1]} average, label = {alphabets[i-1]}")
	plt.ylabel("potential")
	plt.xlabel("time after stimulus")
	for channel in range(len(mat['data'])):
		all = []
		for time in range(len(mat['data'][channel])):
			tmp = sum(mat['data'][channel][time][label_dict[i][0]:label_dict[i][1]])/300
			#norm = np.linalg.norm(tmp)
			#normalized_array = tmp/norm 
			#all.append(normalized_array)
			all.append(tmp)
		all = np.array(all)
		plt.plot(mat['time_points'][:,0], (all-all.min())/(all.max()-all.min()), label=mat['channel_labels'][channel][0][0])
	plt.legend(loc='best')
	plt.show()

In [ ]:
#each alphabet avg with pandas normalization

for i in label_dict:
	plt.figure(figsize=(16,9))
	plt.title(f"sample {label_dict[i][0]} to {label_dict[i][1]} average, label = {alphabets[i-1]}")
	plt.ylabel("potential")
	plt.xlabel("time after stimulus")
	for channel in range(len(mat['data'])):
		all = []
		for time in range(len(mat['data'][channel])):
			tmp = sum(mat['data'][channel][time][label_dict[i][0]:label_dict[i][1]])/300
			#norm = np.linalg.norm(tmp)
			#normalized_array = tmp/norm 
			#all.append(normalized_array)
			all.append(tmp)
		all = np.array(all)
		plt.plot(mat['time_points'][:,0], (all - all.mean()) / all.std(), label=mat['channel_labels'][channel][0][0])
	plt.legend(loc='best')
	plt.show()

In [ ]:
#each alpahbet avg - total avg for each time frame
for i in label_dict:
	plt.figure(figsize=(16,9))
	plt.title(f"sample {label_dict[i][0]} to {label_dict[i][1]} average, label = {alphabets[i-1]}")
	plt.ylabel("potential")
	plt.xlabel("time after stimulus")
	for channel in range(len(mat['data'])):
		all = []
		for time in range(len(mat['data'][channel])):
			all.append(sum(mat['data'][channel][time][label_dict[i][0]:label_dict[i][1]])/300-all_avg[channel][time])
		all = np.array(all)
		plt.plot(mat['time_points'][:,0],(all - all.mean()) / all.std(), label=mat['channel_labels'][channel][0][0])
	plt.legend(loc='best')
	plt.show()

In [ ]:
#NEW
all_dict = {}
#each alphabet avg
for i in label_dict:
    all_dict[alphabets[i-1]] = {}
    for channel in range(len(mat['data'])):
        all = []
        for time in mat['data'][channel]:
            all.append(sum(time[label_dict[i][0]:label_dict[i][1]])/300)
        all_dict[alphabets[i-1]][mat['channel_labels'][channel][0][0]] = all
df = pd.DataFrame.from_dict(all_dict)
        
#print graph
for alpha in ['A','B']:
    plt.figure(figsize=(16,9))
    plt.title(f"label = {alpha}")
    plt.ylabel("potential")
    plt.xlabel("time after stimulus")
    for channel in df[alpha].keys():
        plt.plot(mat['time_points'][:,0],df[alpha][channel], label=channel)
    plt.legend(loc='best')
    plt.show()
    
#print difference graph for P7
base_alpha = 'A'
base_ch = 'P7'
baseline = np.array(df[base_alpha][base_ch])
plt.figure(figsize=(16,9))
plt.title(f"alpha - {base_alpha}, channel = {base_ch}")
plt.ylabel("potential")
plt.xlabel("time after stimulus")
for alpha in df.keys():
    if alpha != base_alpha:
        plt.plot(mat['time_points'][:,0],np.array(df[alpha][base_ch])-baseline, label=alpha)
plt.legend(loc='best')
plt.show()

#print difference graph for A and B
base_alpha = 'B'
alpha = 'A'
baseline = df[base_alpha]
compare = df[alpha]
plt.figure(figsize=(16,9))
plt.title(f"label = {alpha} - {base_alpha}")
plt.ylabel("potential")
plt.xlabel("time after stimulus")
for ch in df[base_alpha].keys():
    plt.plot(mat['time_points'][:,0],np.array(compare[ch])-baseline[ch], label=ch)
plt.legend(loc='best')
plt.show()